In [2]:
import pandas as pd

In [14]:
transactions = pd.read_csv('./data/input/Transactions.csv')
transactions.head()

,Transaction ID,Account Number,Sort Code,Bank
0,3888,62230725,95-98-82,Data Source Bank
1,4746,83172326,42-86-38,Barclays Bank
2,5404,34302539,53-28-21,Barclays Bank
3,9013,13350031,93-87-71,Natwest
4,2535,68745993,57-14-32,Barclays Bank


In [15]:
swift_codes = pd.read_csv('./data/input/Swift Codes.csv')
swift_codes.head()

,Bank,SWIFT code,Check Digits
0,Lloyds Bank,LOYD,C1
1,Barclays Bank,BARC,22
2,Halifax,HLFX,22
3,HSBC,HBUK,4B
4,Natwest,NWBK,2L


# Preprocessing

In [35]:
df = ( transactions
    # Remove the '-' characters from the Sort Code column
    .assign(Sort_Code = lambda x: x['Sort Code'].str.replace('-', ''))
    # Merge the 'swift_codes' table with the 'transactions' table
    .merge(swift_codes, how = 'left', left_on = 'Bank', right_on = 'Bank')
    # Add a field for the 'Country Code' (it will be GB for every row as we only have UK banks)
    .assign(Country_Code = 'GB')
    # Construct the 'IBAN' code column
    .assign(IBAN = lambda x: x['Country_Code'] + x['Check Digits'].astype(str) + x['SWIFT code'].astype(str)  + x['Sort_Code'].astype(str) + x['Account Number'].astype(str))
# Remove unnecessary fields
)[['Transaction ID', 'IBAN']]

df.head()

,Transaction ID,IBAN
0,3888,GB12DSBX95988262230725
1,4746,GB22BARC42863883172326
2,5404,GB22BARC53282134302539
3,9013,GB2LNWBK93877113350031
4,2535,GB22BARC57143268745993


In [37]:
df.to_csv(path_or_buf='./data/output/output_2023_02.csv', index = False, quoting=1, quotechar='"', sep=';')